In [1]:
from pandas import read_csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [2]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# Vizualize RNN with toy example

demo_model = create_RNN(2, 1, (3,1), activation=['linear', 'linear'])

wx = demo_model.get_weights()[0]
wh = demo_model.get_weights()[1]
bh = demo_model.get_weights()[2]
wy = demo_model.get_weights()[3]
by = demo_model.get_weights()[4]

print('wx = ', wx, ' wh = ', wh, ' bh = ', bh, ' wy =', wy, 'by = ', by)

x = np.array([1, 2, 3])
# Reshape the input to the required sample_size x time_steps x features 
x_input = np.reshape(x,(1, 3, 1))
y_pred_model = demo_model.predict(x_input)


m = 2
h0 = np.zeros(m)
h1 = np.dot(x[0], wx) + h0 + bh
h2 = np.dot(x[1], wx) + np.dot(h1,wh) + bh
h3 = np.dot(x[2], wx) + np.dot(h2,wh) + bh
o3 = np.dot(h3, wy) + by

print('h1 = ', h1,'h2 = ', h2,'h3 = ', h3)

print("Prediction from network ", y_pred_model)
print("Prediction from our computation ", o3)

In [3]:
# Parameter split_percent defines the ratio of training examples
def get_train_test(url, split_percent=0.8):
    df = read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data

sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, data = get_train_test(sunspots_url)

In [19]:
df = read_csv(sunspots_url, usecols=[1], engine='python')
data = np.array(df.values.astype('float32'))

In [10]:
# Prepare the input X and target Y
def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]
    # Prepare X
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 1))    
    return X, Y

time_steps = 12
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)

In [16]:
model = create_RNN(hidden_units=3, dense_units=1, input_shape=(time_steps,1), 
                   activation=['tanh', 'tanh'])
model.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

Epoch 1/20
187/187 - 1s - loss: 0.0266 - 645ms/epoch - 3ms/step
Epoch 2/20
187/187 - 0s - loss: 0.0113 - 348ms/epoch - 2ms/step
Epoch 3/20
187/187 - 0s - loss: 0.0080 - 259ms/epoch - 1ms/step
Epoch 4/20
187/187 - 0s - loss: 0.0066 - 199ms/epoch - 1ms/step
Epoch 5/20
187/187 - 0s - loss: 0.0057 - 199ms/epoch - 1ms/step
Epoch 6/20
187/187 - 0s - loss: 0.0052 - 202ms/epoch - 1ms/step
Epoch 7/20
187/187 - 0s - loss: 0.0048 - 195ms/epoch - 1ms/step
Epoch 8/20
187/187 - 0s - loss: 0.0045 - 198ms/epoch - 1ms/step
Epoch 9/20
187/187 - 0s - loss: 0.0043 - 203ms/epoch - 1ms/step
Epoch 10/20
187/187 - 0s - loss: 0.0042 - 362ms/epoch - 2ms/step
Epoch 11/20
187/187 - 0s - loss: 0.0040 - 279ms/epoch - 1ms/step
Epoch 12/20
187/187 - 0s - loss: 0.0040 - 210ms/epoch - 1ms/step
Epoch 13/20
187/187 - 0s - loss: 0.0038 - 202ms/epoch - 1ms/step
Epoch 14/20
187/187 - 0s - loss: 0.0038 - 201ms/epoch - 1ms/step
Epoch 15/20
187/187 - 0s - loss: 0.0037 - 307ms/epoch - 2ms/step
Epoch 16/20
187/187 - 0s - loss: 0

In [17]:
def print_error(trainY, testY, train_predict, test_predict):    
    # Error of predictions
    train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
    # Print RMSE
    print('Train RMSE: %.3f RMSE' % (train_rmse))
    print('Test RMSE: %.3f RMSE' % (test_rmse))    

# make predictions
train_predict = model.predict(trainX)
test_predict = model.predict(testX)
# Mean square error
print_error(trainY, testY, train_predict, test_predict)

2/2 [==============================] - 0s 2ms/step
Train RMSE: 0.058 RMSE
Test RMSE: 0.085 RMSE
